In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-13 01:08:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-11-13 01:08:34 (5.99 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"

spark.sparkContext.addFile(url)

videogame_df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep="\t")

videogame_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [ ]:
videogame_df.count()

1785997

In [ ]:
videogame_df = videogame_df.dropDuplicates()

In [ ]:
videogame_df = videogame_df.dropna()

In [ ]:
review_id_df = videogame_df.select('review_id', 'customer_id', 'product_id', 'product_parent', 'review_date')

In [ ]:
review_id_df.show(1)

+-------------+-----------+----------+--------------+-----------+
|    review_id|customer_id|product_id|product_parent|review_date|
+-------------+-----------+----------+--------------+-----------+
|RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
+-------------+-----------+----------+--------------+-----------+
only showing top 1 row



In [ ]:
from pyspark.sql.functions import *

review_id_df = review_id_df.select('review_id', 'customer_id', 'product_id', 'product_parent', 'review_date',to_date('review_date', 'yyyy-mm-dd').alias('review_date_new'))

In [ ]:
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+---------------+
|     review_id|customer_id|product_id|product_parent|review_date|review_date_new|
+--------------+-----------+----------+--------------+-----------+---------------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|     2015-01-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|     2015-01-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|     2015-01-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|     2015-01-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|     2015-01-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|     2015-01-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|     2015-01-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|     2015-01-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|     2015-01-31|
| RN

In [ ]:
 review_id_df = review_id_df.drop(col("review_date")).printSchema()

AnalysisException: ignored

In [ ]:
review_id_df2.show(1)

AttributeError: ignored

In [ ]:
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+------------+
|     review_id|customer_id|product_id|product_parent|review_date|review_date1|
+--------------+-----------+----------+--------------+-----------+------------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|  2015-01-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|  2015-01-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|  2015-01-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|  2015-01-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|  2015-01-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|  2015-01-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|  2015-01-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|  2015-01-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|  2015-01-31|
| RNQOY62705W1K|   48467989|B0000AV7GB| 

In [ ]:
 review_id_df = review_id_df.select(col("review_date1").alias("review_date")

SyntaxError: ignored

In [ ]:
from pyspark.sql.functions import *
review_id_df = review_id_df

SyntaxError: ignored

In [ ]:
review_id_df.show(10)

+-----------+----------+
|review_date|      date|
+-----------+----------+
| 2013-03-28|2013-01-28|
| 2013-10-28|2013-01-28|
| 2013-12-31|2013-01-31|
| 2014-03-21|2014-01-21|
| 2007-11-29|2007-01-29|
| 2011-11-06|2011-01-06|
| 2013-10-07|2013-01-07|
| 2010-12-28|2010-01-28|
| 2008-02-28|2008-01-28|
| 2013-02-04|2013-01-04|
+-----------+----------+
only showing top 10 rows



In [ ]:
products_df = videogame_df.select('product_id', 'product_title')
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004HX5OFW|Xbox 360 Slim AC ...|
|B000QW9D14|Blue Dragon - Xbo...|
|B00D3IKP7Y|        Pokemon Ruby|
|B0094X28J0|Nintendo Wii Remo...|
|B000UQAUWW|Xbox 360 Console ...|
|B004S5PBM0|           Rocksmith|
|B002EE7OKE|Sin and Punishmen...|
|B003R7H5TC|Need for Speed: H...|
|B000R3BNDI|Lego Star Wars: T...|
|B000NKFEZI|   Slingo Quest - PC|
+----------+--------------------+
only showing top 10 rows



In [ ]:
customers_df = videogame_df.select("customer_id", "review_id")
customers_df.show(10)

+-----------+--------------+
|customer_id|     review_id|
+-----------+--------------+
|   26197704|R10010D6B1QM7U|
|   10409381|R100BI61LIKH2N|
|   39071377|R100TPJCYDYGBU|
|   32826337|R1018OO9U6UUWO|
|   16385120|R102YIWZEIAXT9|
|   14198390|R1034S7FA21OI1|
|   16965219|R103LC8CTAHWZ6|
|   48295356|R103PT7S4HRIP8|
|   21497984|R104RCQZYPYDXS|
|   22595160|R105KZIXHZW63W|
+-----------+--------------+
only showing top 10 rows



In [ ]:
customers_group_df = customers_df.groupBy("customer_id").count()
customers_group_df.show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   40504830|    8|
|    3675423|    3|
|   11393715|    9|
|   14175902|   22|
|   39664150|    4|
|   14534785|    1|
|   47765241|    7|
|   33929721|    1|
|   41446882|    8|
|   52714119|    4|
+-----------+-----+
only showing top 10 rows



In [ ]:
customers_group_df = customers_group_df.withColumnRenamed("count", "customer_count")
customers_group_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   40504830|             8|
|    3675423|             3|
|   11393715|             9|
|   14175902|            22|
|   39664150|             4|
|   14534785|             1|
|   47765241|             7|
|   33929721|             1|
|   41446882|             8|
|   52714119|             4|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://alpha-v1.cprlsbrw9ixr.us-east-1.rds.amazonaws.com:5432/big-data-challenge"
config = {"user":"postgres", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}


In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

KeyboardInterrupt: ignored

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
customers_group_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)